In [1]:
import numpy as np

def jacobi(A, b, x0=None, tol=1e-10, max_iter=100_000, return_history=False):
    
    A = np.array(A, dtype=float)
    b = np.array(b, dtype=float).reshape(-1)

    n = A.shape[0]
    if A.shape != (n, n):
        raise ValueError("A must be square.")
    if b.shape[0] != n:
        raise ValueError("b must have length n.")

    if x0 is None:
        x = np.zeros(n, dtype=float)
    else:
        x = np.array(x0, dtype=float).reshape(-1)
        if x.shape[0] != n:
            raise ValueError("x0 must have length n.")

    D = np.diag(A)
    if np.any(D == 0):
        raise ZeroDivisionError("Jacobi requires all diagonal entries A[i,i] nonzero.")
    R = A - np.diagflat(D)

    history = [x.copy()] if return_history else None

    converged = False
    for k in range(1, max_iter + 1):
        x_new = (b - R @ x) / D

        diff_inf = np.linalg.norm(x_new - x, ord=np.inf)
        x_inf = np.linalg.norm(x_new, ord=np.inf)
        if diff_inf <= tol * (1.0 + x_inf):
            x = x_new
            converged = True
            if return_history:
                history.append(x.copy())
            break

        x = x_new
        if return_history:
            history.append(x.copy())

    residual_inf = np.linalg.norm(b - A @ x, ord=np.inf)

    if return_history:
        return x, k, converged, residual_inf, history
    return x, k, converged, residual_inf


def gauss_seidel(A, b, x0=None, tol=1e-10, max_iter=100_000, return_history=False):

    A = np.array(A, dtype=float)
    b = np.array(b, dtype=float).reshape(-1)

    n = A.shape[0]
    if A.shape != (n, n):
        raise ValueError("A must be square.")
    if b.shape[0] != n:
        raise ValueError("b must have length n.")

    if x0 is None:
        x = np.zeros(n, dtype=float)
    else:
        x = np.array(x0, dtype=float).reshape(-1)
        if x.shape[0] != n:
            raise ValueError("x0 must have length n.")

    if np.any(np.diag(A) == 0):
        raise ZeroDivisionError("Gauss-Seidel requires all diagonal entries A[i,i] nonzero.")

    history = [x.copy()] if return_history else None

    converged = False
    for k in range(1, max_iter + 1):
        x_old = x.copy()

        for i in range(n):
            # sum_{j<i} Aij*xj (new values) + sum_{j>i} Aij*xj (old values)
            s1 = A[i, :i] @ x[:i]
            s2 = A[i, i+1:] @ x_old[i+1:]
            x[i] = (b[i] - s1 - s2) / A[i, i]

        diff_inf = np.linalg.norm(x - x_old, ord=np.inf)
        x_inf = np.linalg.norm(x, ord=np.inf)
        if diff_inf <= tol * (1.0 + x_inf):
            converged = True
            if return_history:
                history.append(x.copy())
            break

        if return_history:
            history.append(x.copy())

    residual_inf = np.linalg.norm(b - A @ x, ord=np.inf)

    if return_history:
        return x, k, converged, residual_inf, history
    return x, k, converged, residual_inf


# Run the three required problems

# Problem 1
A1 = np.array([
    [0.582745, 0.48    , 0.10    , 0.     , 0.     ],
    [0.48    , 1.044129, 0.46    , 0.10   , 0.     ],
    [0.10    , 0.46    , 1.10431 , 0.44   , 0.10   ],
    [0.      , 0.10    , 0.44    , 0.963889, 0.42  ],
    [0.      , 0.      , 0.10    , 0.42   , 0.522565]
], dtype=float)
b1 = np.array([1.162745, 2.084129, 2.20431, 1.923889, 1.042565], dtype=float)

# Problem 2
A2 = np.array([[1, 1], [1, 1.0001]], dtype=float)
b2 = np.array([1, 1], dtype=float)

# Problem 3
A3 = np.array([[1, 1], [1, 1.0001]], dtype=float)
b3 = np.array([1, 1.0001], dtype=float)

tol = 1e-10
max_iter = 200_000

for idx, (A, b) in enumerate([(A1, b1), (A2, b2), (A3, b3)], start=1):
    xJ, kJ, convJ, rJ = jacobi(A, b, tol=tol, max_iter=max_iter)
    xG, kG, convG, rG = gauss_seidel(A, b, tol=tol, max_iter=max_iter)

    print(f"\n--- Problem {idx} ---")
    print("Jacobi:")
    print("  converged:", convJ)
    print("  iterations:", kJ)
    print("  x:", xJ)
    print("  ||b-Ax||_inf:", rJ)

    print("Gauss-Seidel:")
    print("  converged:", convG)
    print("  iterations:", kG)
    print("  x:", xG)
    print("  ||b-Ax||_inf:", rG)


--- Problem 1 ---
Jacobi:
  converged: True
  iterations: 5496
  x: [1. 1. 1. 1. 1.]
  ||b-Ax||_inf: 2.191917758409545e-10
Gauss-Seidel:
  converged: True
  iterations: 32
  x: [1. 1. 1. 1. 1.]
  ||b-Ax||_inf: 4.221845095742083e-11

--- Problem 2 ---
Jacobi:
  converged: False
  iterations: 200000
  x: [0.99995458 0.        ]
  ||b-Ax||_inf: 4.5422633894176556e-05
Gauss-Seidel:
  converged: True
  iterations: 2
  x: [1. 0.]
  ||b-Ax||_inf: 0.0

--- Problem 3 ---
Jacobi:
  converged: False
  iterations: 200000
  x: [0.         0.99995458]
  ||b-Ax||_inf: 4.5427176185386386e-05
Gauss-Seidel:
  converged: True
  iterations: 131232
  x: [1.999839e-06 9.999980e-01]
  ||b-Ax||_inf: 1.9996404532207634e-10
